# Module 5, Week 1 In Class Exercise

Probability distributions

**Before class reading: Chapters 9.3, 9.5, 10.0, and 10.2 from Data8 textbook **

**Last week we:**
- Load peak ground acceleration observations from two notable M6 quakes in California
- Fit a ground motion prediction equation (GMPE) using `polyfit`
- Vary our assumed mean event depth to find better fitting model
- Fit a GMPE after weighing the data by the distance distribution using `linalg.solve`

**Our goals for today:**
- Review some key statistics topics: samples versus populations and empirical versus theorectical distributions
- Simulate a head/tail coin toss i.e. Binomial distribution
- Simulate cars passing a point i.e. Poisson distribution
- Simulate geomagnetic polarity reversals i.e. Gamma distribution


## Setup

Run this cell as it is to setup your environment.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as scipy
import scipy.stats as sps

## Basic statistical concepts

So, what is statistics?  Statistics is the way we analyze, interpret and model data. To do it properly we need to understand a few concepts: 

1) **Population** versus **sample**: the _population_ is the set of all possible outcomes of a given measurement (if you had an infinite number of data points), while the _sample_ is what you have - a finite number of data points.  

2) **Probability**: the measure of how likely it is for a particular event to occur.  If something is impossible, it has a probability $P$ of 0.  If it is a certainty, it has a probability $P$ of 1.  

3) **Theoretical** versus **empirical** distributions: Empirical distributions are measured data. Theoretical distributions are analytical probabililty functions described with an equation.  These can be applied to data, allowing interpretations about the likelihood of observing a given measurement.  

 
Additional resources: Olea (2008): https://pubs.usgs.gov/of/2008/1017/ofr2008-1017_rev.pdf; Davis, J. (2002): Statistical and Data Analysis in Geology, 3rd Ed, Wiley, Hoboken. 


There are many different types of probability distributions and evaluating the equations gives us a theoretical distribution. 

<img src="./paranormal_dist.png">

Samples are finite collections of observations which belong to a distribution. In this exercise, we will simulate 'measurements' by drawing 'observations' from a theoretical distribution.  This is the _Monte Carlo_ approach (after the gambling town).

Examples of theoretical versus empirical distributions:

### Binomial distribution:

#### Theoretical

Perhaps the most straight forward distribution is the _binomial_ distribution which describes the probability of a particular outcome when there are only two possibilities (yes or no, heads or tails, 1 or 0).   For example, in a coin toss experiment (heads or tails), if we flip the coin  $n$ times, what is the probability of getting $x$ 'heads'?  We assume that the probability $p$ of a head for any given coin toss is 50%; put another way $p$ = 0.5.  

The binomial distribution can be described by an equation: 

$$P=f(x,p,n)= \frac{n!}{x!(n-x)!}p^x(1-p)^{n-x}$$

We can look at this kind of distribution by evaluating the probability for getting $x$ 'heads' out of $n$ attempts. We'll code the equation as a function, and calculate the probability $P$ of a particular outcome (e.g., $x$ heads in $n$ attempts). Note that for a coin toss, $p$ is 0.5, but other yes/no questions can be investigated as well (e.g., chance of winning the lottery given purchase of $n$ tickets). 

In [ ]:
def Binomial_probability(x,p,n):
    """
    This function computes the probability of getting x particular outcomes (heads) in n attempts, where p is the 
    probability of a particular outcome (head) for any given attempt (coin toss).
    """
    # initialize size of probability array
    prob = np.zeros(len(x)) 
    
    # compute the binomial probability of getting x outcomes in n attempts
    for i in np.arange(0,len(x),1):
        prob[i] = (np.math.factorial(...)*(p**(...[i]))*(1.0-p)**(...[i])/  \
                   (np.math.factorial(...[i])* np.math.factorial(...[i])))
    
    #return the output
    return prob


We can use `Binomial_probability` to look at the predicted likelihood of getting $x$ heads out of $n=12$ attempts (coin tosses) with a $p$ (probability) of 0.5. 

In [ ]:
# calculate the probability  of x heads in n attempts with a probability of 0.5
n=12 # number of attempts in each trial
p=0.5 # probability of getting a head
xs=np.arange(...,...,...) # range of test 'head' values from 0 to n steps of 1
Probability=Binomial_probability(...,...,...) # probability of getting x heads out of n attempts
plt.bar(xs,Probability,width=.5,color='cyan') # plot as bar plot
plt.plot(xs,Probability,'r-',linewidth=2) # plot as solid line
plt.xlabel('Number of heads out of $n$ attempts') # add labels
plt.ylabel('Probability') 


plt.text(1,.2, 'n = %i'%(n),  fontsize=14) # place a note in upper left 
plt.title('Binomial Distribution');

_Red line is the theoretical probability density function._

Although other outcomes can very well occur, what is the most probable number of heads you'll observe in 12 coin tosses? What is the probability of this number of heads?

_Write your answer here._

#### Empirical

One great features of computers is that we can simulate a data sample to compare to our theoretical predictions. 
We can use the module `numpy.random` to generate examples of simulated data sets in a process called _Monte Carlo simulation_. 

In [ ]:
help(np.random.binomial)

To generate some  data, you could either patiently do the experiment with a coin toss, or we can just use the `np.random.binomial` function to simulate 'realistic' data.  

`np.random.binomial( )` requires 2 parameters, $n$ and $p$, with an optional keyword argument `size` (if `size` is not specified, it returns a single trial).   Each call to `np.random.binomial( )` returns the number of heads flipped in a single trial of $n$ coin tosses, given the probablity $p=0.5$. 

Let's try it with $n=12$ and $p=0.5$.  The following code block returns the number of heads out of $n$ attempts. You will get a different answer every time you run it. 

In [ ]:
n = 12
p = 0.5 # same as before
x = np.random.binomial(...,...) # the number of heads over the n tosses

print (x, 'heads')

 As the number of times you repeat this 'experiment' approaches infinity, the distribution of outcomes will approach the theoretical distribution (i.e., you will get an average of 9 heads out of 12 attempts 5% of the time).   

So let's compare the results simulated via Monte Carlo for some number of experiments ($Nmc$) with the theoretical distribution.  To do this,  we pretend that each student in the class ($Nmc=18$) flips a coin $n=12$ times and reports the number of heads.  We can collect the number of heads flipped by each student in the variable `Simulated`

In [ ]:
n = 12
p = 0.5 # same as before
Nmc=18 # number of simulated experiments each with n  attempts
Simulated=np.random.binomial(...,...,size=...)

# plot
plt.bar(...,...,color='cyan', label='Theoretical') # theoretical curve as bar graph
plt.hist(...,density=True,color='orange',histtype='step',linewidth=3, label='Simulated') # simulated data
plt.xlabel('Number of heads out of $n$ attempts')
plt.ylabel('Fraction of simulated experiments')
plt.text(1,.3, 'n = %i'%(n),  fontsize=14)
plt.legend();

_Orange line is the Monte Carlo results.  Blue line is the theoretical probability density function._

Notice that every time you repeat this, the Monte Carlo results are a little different.  And if you change $Nmc$ to be, say 10, you get more and more "weird" results.  But if you set $Nmc$ to 5000, your results would look consistently  closer to the theoretical prediction. 


### Poisson distribution:

#### Theoretical

The Poisson distribution gives the probability that an event (with two possible outcomes) occurs $k$ number of times in an interval of time where $\lambda$ is the expected rate of occurance. The Poisson distribution is the limit of the binomial distribution for large $n$. So if you take the limit of the binomial distribution as $n \rightarrow \infty$

$\lim_{n \rightarrow \infty } \frac{n!}{k!(n-k)!}p^k(1-p)^{n-k} = \lim_{n \rightarrow \infty } \frac{n!}{k!(n-k)!}(\frac{\lambda}{n})^k(1-\frac{\lambda}{n})^{n-k} $

you'll get the Poisson distribution:

$P(k$ events in interval$) = e^{-\lambda}\frac{\lambda^{k}}{k!}$

Because $\lim_{x \rightarrow \infty } (1+\frac{a}{x})^{x}=e^{a}$ and $\lim_{x \rightarrow \infty } \frac{x!}{x^{k}(x-k)!} = 1$

In [ ]:
def Poisson_probability(k,lam):
    """
    This function computes the probability of getting x particular outcomes (heads) in n attempts, where p is the 
    probability of a particular outcome (head) for any given attempt (coin toss).
    """
    
    # compute the poisson probability of getting k outcomes when the expected rate is lam
    prob = (np.xxx(-1*...))*(...**...)/np.math.factorial(...)
    
    #return the output
    return prob

For example if you are watching a section of road and counting the number of cars that pass. If the expected rate of cars going by is 10 cars per hour, and you observe for some huge number of hours (so $n \approx \infty$) then the probability of observing 12 cars in an hour is:

In [ ]:
lam = 10;
k = 12;
prob = Poisson_probability(...,...);
print (prob)

The shape of the theoretical Poisson distribution depends on the value of $\lambda$ of the Poisson process being observed.

In [ ]:
num_events = np.arange(0,20,1)

# lambda = 1
P1 = np.zeros(len(num_events))
for i in np.arange(0,20,1):
    P1[i] = Poisson_probability(...,...)

# lambda = 4
P4 = np.zeros(len(num_events))
for i in np.arange(0,20,1):
    P4[i] = Poisson_probability(i,...)
    
# lambda = 10
P10 = np.zeros(len(num_events))
for i in np.arange(0,20,1):
    P10[i] = Poisson_probability(i,...)
    
# plot    
plt.plot(num_events,...,'o-',label='$\lambda$ = 1') # theoretical curve for lam = 1
plt.plot(num_events,...,'o-',label='$\lambda$ = 4') # theoretical curve for lam = 4
plt.plot(num_events,...,'o-',label='$\lambda$ = 10') # theoretical curve for lam = 10
plt.xlabel('k')
plt.ylabel('Probability')
plt.title('Poisson Distribution');
plt.legend();

#### Empirical

Again we'll use the module `numpy.random` to generate examples of simulated data sets.

In [ ]:
help(np.random.poisson)

To generate some  data, you could either patiently do the experiment watching cars or we can just use the `np.random.poisson` function to simulate 'realistic' data.  

`np.random.poisson( )` requires 1 parameter `lam` and an optional parameter `size`.  Each call to `np.random.poisson( )` returns `size` number of draws from a Poisson distribution with $\lambda =$ `lam`.

Let's try it with $\lambda = 10$ and $p=0.5$.  You will get a different answer every time you run the following: 

In [ ]:
lam = 10

# theoretical poisson prob distribution
num_events = np.arange(0,lam*2,1)
P = np.zeros(len(num_events))
for i in np.arange(0,lam*2,1):
    P[i] = Poisson_probability(i,lam)

# empirical simulated poisson prob distribution
Nmc=5000 # number of simulated experiments
Simulated=np.random.poisson(...,size=...)

# plot
plt.bar(num_events,...,color='cyan', label='Theoretical') # theoretical curve as bar graph
plt.hist(...,density=True,color='orange',histtype='step',linewidth=3, label='Simulated') # simulated sample
plt.xlabel('k Number of events')
plt.ylabel('Probability')
plt.title('Poisson Distribution');
plt.text(1,.08, '$\lambda$ = %i'%(lam),  fontsize=14)
plt.legend();

### The occurance of geomagnetic reversals can be described by a Poisson distribution.

The Geomagnetic Polarity Time Scale is the record of geomagnetic reversals over the past ~150 million years. During the past ~40 Myr the reversal process has been stationary, meaning that the $k$ and $\lambda$ parameters did not change with time.

<img src="./GPTS.png" width = 600>

> Source: Gee and Kent (2007) "Source of Oceanic Magnetic Anomalies and the Geomagnetic Polarity Timescale"

Load the GPTS, ages of reversal events:

In [ ]:
data=pd.read_table('age_polarity_1.txt')
data.head(10)

In [ ]:
age = data.Time_Myr
polarity = data.Polarity

plt.figure(1,(20,2))
plt.plot(...,...);
plt.xlim(0, age[319])
plt.xlabel('Age, Myr')
plt.ylabel('Polarity')
plt.title('Geomagnetic Polarity Time Scale');

In [ ]:
# plot the GPTS to look like the above figure with plt.fill
# to do that we need to add an extra -1 at the start and end of polarity i.e. at zero and final age (41.257 Myr)
fill_age = np.hstack(([0], age, age[319]))
fill_polarity = np.hstack(([-1], polarity, [-1]))

plt.figure(1,(20,2))
plt.xxx(fill_age,fill_polarity);
plt.xlim(0, age[319])
plt.xlabel('Age, Myr')
plt.ylabel('Polarity, Blue = Normal')
plt.title('Geomagnetic Polarity Time Scale');

### Gamma distribution:

#### Theoretical

The Gamma distribution gives the probability of a waiting time between Poisson distributed events.

Consider the distribution function $D(x)$ of waiting times until the $h$th Poisson event given a Poisson distribution with a rate of change $\lambda$,

$$ D(x) = P (X \le x) = 1 - P(X > x) = 1-\sum_{k=0}^{h-1}\frac{(\lambda x)^{k}e^{-\lambda x}}{k!} = 1-e^{-\lambda x}\sum_{k=0}^{h-1}\frac{(\lambda x)^{k}}{k!} = 1-\frac{\Gamma(h,x\lambda) }{\Gamma (h)}$$ 

where $\Gamma (x) = (x-1)!$ is a complete gamma function and $\Gamma (n,x) = (n-1)! e^{-x}\sum_{k=0}^{n-1}\frac{x^{k}}{k!}$ an incomplete gamma function. The corresponding probability function $P(x)$ of waiting times until the $h$th Poisson event is then obtained by differentiating  $D(x)$,

$$ P(x) = D'(x) = \frac{\lambda (\lambda x)^{h-1}}{(h-1)!}e^{-\lambda x} $$

Now let $\alpha=h$ (not necessarily an integer) and define $\theta=1/\lambda$ to be the time between changes. Then the above equation can be written

$$ P(x) = \frac{x^{\alpha-1}e^{-x/\theta}}{\Gamma (\alpha) \theta^{\alpha}} $$

which is the probability of a duration time $x$ between events.


We need to compute $\theta$ the expected time between polarity changes. $\theta = \mu / \alpha$ where $\mu$ is the average chron duration. A value for $\alpha$ greater than one can be interpreted either as an artefact linked to some short intervals missing in the GPTS or to some short term memory within the dynamo that would inhibit a second reversal just after a first one has occurred.

<img src="./alpha_greater_one.png" width = 600>

> Source: McFadden (1984) "Statistical Tools for the Analysis of Geomagnetic Reversal Sequence"

In [ ]:
chron_duration = age.values[1:]-age.values[:-1]
chron_duration = chron_duration[chron_duration != 0.0]

avg_dur = np.mean(...)
print('Over the past 40 Myr, the average time between reversals is %4.3f' %avg_dur, 'Myr.')

The most recent reversal happened 0.780 Myr ago. Are we overdue for a geomagnetic reversal?!

Define a function for the Gamma probability distribution:
$$ P(x) = \frac{x^{\alpha-1}e^{-x/\theta}}{\Gamma (\alpha) \theta^{\alpha}} $$

In [ ]:
def Gamma_probability(x,theta,alpha):
    """
    This function computes the probability waiting x time between poisson events (polarity change), 
    given theta the expected time between changes and alpha the shape parameter for the gamma distribution
    """
    
    # compute the gamma probability of getting a waiting time of x between events
    prob = (x**(...) * np.exp(...)) / (scipy.special.gamma(...)* ...**...)
    
    #return the output
    return prob

What is the probability of having a chron with a 0.78 Myr duration?

In [ ]:
x = 0.780;
alpha = 1.2; # mean of estimate from McFadden (1984) 
theta = avg_dur / alpha;

current_chron_prob = Gamma_probability(...,...,...)
print('The probability of a 0.780 Myr chron (i.e. the Bruhnes) is: %4.3f.'% current_chron_prob)

#### Empirical

The observed GPTS gives us one realization of an empirical distribution.

In [ ]:
b=np.arange(0.,3.,0.1);
plt.hist(...,bins=b,density=True,color='green',histtype='step',linewidth=3, label='Observed');
plt.xlabel('Chron Duration')
plt.title('Geomagnetic Polarity Time Scale 0-41 Myr');
plt.legend();

In [ ]:
# theoretical gamma prob distribution
chrons = np.arange(0.0,3.0,0.01)
gamma_P = np.zeros(len(chrons))
for i in np.arange(0,len(chrons),1):
    gamma_P[i] = Gamma_probability(chrons[i],theta,alpha) # from above alpha = 1.2 and  theta = avg_dur / alpha


# plot
plt.plot(...,...,color='cyan', label='Theoretical') # theoretical curve 
plt.hist(...,bins=b,density=True,color='green',histtype='step',linewidth=3, label='Observed') # observed durations 
plt.xlabel('Polarity Chron Duration')
plt.ylabel('Probability')
plt.title('Gamma Distribution');
plt.legend();



We can use the function `np.random.gamma` to simulate additional empirical distributions.

In [ ]:
help(np.random.gamma)

We'll use the `np.random.gamma` function to simulate 'realistic' data.  

`np.random.gamma( )`  has 2 specified parameters: `shape` (sometimes designated "$\alpha$") and `scale` (sometimes designated "$\theta$"), where both parameters are > 0, and  an optional keyword argument `size` (if `size` is not specified, it returns a single trial). Each call to `np.random.gamma( )` returns a chron duration pulled from the gamma distribution.


In [ ]:
# empirical simulated gamma prob distribution
# from above alpha = 1.2 and  theta = avg_dur / alpha
random_chron_dur = np.random.gamma(..., ..., len(chron_duration)-1); 


# plot
plt.plot(...,...,color='cyan', label='Theoretical') # theoretical curve 
plt.hist(...,bins=b,density=True,color='green',histtype='step',linewidth=3, label='Observed') # observed durations 
plt.hist(...,bins=b,density=True,color='orange',histtype='step',linewidth=3, label='Simulated') # simulated durations
plt.xlabel('Polarity Chron Duration')
plt.ylabel('Probability')
plt.title('Gamma Distribution');
plt.legend();

Let's plot this new random polarity time scale!

In [ ]:
# construct the polarity time scale from the chron durations 
A = np.repeat(np.cumsum(random_chron_dur), 2); # add up durations to get ages and repeat for 1 and -1 polarities
random_GPTS = np.zeros(len(A)+2);
random_GPTS[1:-1] = A;
random_GPTS[-1] = A[-1]+1.0;
fill_random_GPTS = np.hstack(([0], random_GPTS, random_GPTS[319])) # add extra points to plot with plt.fill

# plot with plt.fill
plt.figure(1,(20,2))
plt.xxx(fill_random_GPTS,fill_polarity);
plt.xlim(0, random_GPTS[319])
plt.xlabel('Age, Myr')
plt.ylabel('Polarity')
plt.title('Simulated Random Geomagnetic Polarity Time Scale');

### Will the field reverse soon?!
But what we _really_ would like to know is how likely is it that a polarity reversal will happen soon. The current normal chron has been going on for 0.78 Myr. To find the probability that a reversal will happen in the next 10 thousand years we need to find that probability of a chron that is longer than 0.78 Myr but shorter than 0.79 Myr. 
$$P (0.78 \le X \le 0.79) = P(X \le 0.79) - P(X \le 0.78) = (1 - P(0.79)) - (1 - P(0.78))$$

In [ ]:
# from above alpha = 1.2 and  theta = avg_dur / alpha
soon = ...; #10kyr = 0.01Myr
next_rev_prob = (1-Gamma_probability(...,...,...)) - (1-Gamma_probability(...,...,...))
print('The probability of a reversal in the next %4.3f Myr: %5.4f' % (soon, next_rev_prob))

So it's pretty unlikely, thankfully!